In [50]:
from dask.distributed import WorkerPlugin, get_worker, LocalCluster, Client, Variable
import time
from datetime import datetime, timedelta

In [2]:
cluster = LocalCluster(
        n_workers=1,
        threads_per_worker=1,
        processes=True,
        # memory_limit="2GB",
    )
client = Client(cluster)
client.forward_logging()

{'tcp://127.0.0.1:60732': {'status': 'OK'}}

In [39]:
def dummy_task(n, var_st):
    print(f"task {n} begins...")
    st_dict = var_st.get()
    st_dict["start_time"] = datetime.now()
    var_st.set(st_dict)
    time.sleep(300)
    print(f"task {n} finished")

In [40]:
vars = []
future_dict = {}

symbol = 1
var_st = Variable()
var_st.set({"symbol":symbol})
vars.append(var_st)
future_dict[symbol] = client.submit(dummy_task, symbol, var_st)

In [41]:
symbol = 2
var_st = Variable()
var_st.set({"symbol": symbol})
vars.append(var_st)
future_dict[symbol] = client.submit(dummy_task, symbol, var_st)

In [42]:
symbol = 3
var_st = Variable()
var_st.set({"symbol": symbol})
vars.append(var_st)
future_dict[symbol] = client.submit(dummy_task, symbol, var_st)

In [79]:
for v in vars:
    try:
        vd = v.get("200ms")
        print(vd)
        if not "start_time" in vd:
            continue
        if vd["start_time"] + timedelta(seconds=60) <= datetime.now():
            ## the task is timeout. check future status. if it's still processing, cancel it.
            symbol = vd["symbol"]
            future = future_dict[symbol]
            if not future.done():
                print(f"cancelling task {symbol}")
                future.cancel()
            v.delete()
            vars.remove(v)
    except TimeoutError as e:
        print('normal timeout')
        vars.remove(v)

task 3 finished
